In [13]:
import json
import os
from src.extractor import Extractor

template_path: str = os.path.join("src", "templates", "barclays_student_template.json")
pdf_path: str = os.path.join("data", "bank_statements", "barclays_student", "pdf", "barclays_student_march.pdf")
pdf_data_path: str = os.path.join("src", "pdf_data", "barclays_student_march_pdf_data.json")
output_path: str = os.path.join("src", "outputs", "barclays_student_march_output.json")

with open(pdf_path, "rb") as pdf_file:
    text_extractor = Extractor(pdf_file.read())
    extracted_data = text_extractor.extract_text()

    with open(pdf_data_path, "w") as f:
        json.dump(extracted_data, f)

template = json.load(open(template_path))
pdf_data = json.load(open(pdf_data_path))

output_data = json.load(open(output_path))


from src.parser import Parser, TableSplitter
from src.pdf_utils import ImageDrawer

parser = Parser()

page_number = 2

table_splitter = TableSplitter(template, parser)

page_content = pdf_data["pages"][page_number - 1]

template = json.load(open(template_path))

def get_column_coordinates_by_field_name(table_rule, field_name):
    for column in table_rule["config"]["columns"]:
        if column["field_name"] == field_name:
            return column["coordinates"]

            

for rule in template["rules"][-1:]:
    if rule["type"] == "table":
        table_rule = rule
        print(table_rule)
        for column in table_rule["config"]["columns"][1:2]:
            print(column)
            coordinates = column["coordinates"]
            delimiter_field_name = "description"

            delimiter_type = "line"

            if delimiter_type == "delimiter":

                delimiter_coordinates = get_column_coordinates_by_field_name(table_rule, delimiter_field_name)
                
                lines_y_coordinates = table_splitter.split_table(
                delimiter_type, page_content, delimiter_coordinates
            )

            if delimiter_type == "line":

                delimiter_coordinates = get_column_coordinates_by_field_name(table_rule, delimiter_field_name)

                lines_y_coordinates = table_splitter.split_table(
                delimiter_type, page_content, delimiter_coordinates
            )   
                print(lines_y_coordinates)
            
            image_with_lines = ImageDrawer.draw_column_box_and_lines(pdf_path, lines_y_coordinates, coordinates, page_number)
            image_with_lines.show()




{'rule_id': 'transactions_page_2_onwards', 'type': 'table', 'config': {'columns': [{'field_name': 'date', 'coordinates': {'top_left': {'x': 0.097, 'y': 0.488}, 'bottom_right': {'x': 0.149, 'y': 0.898}}, 'type': 'text'}, {'field_name': 'description', 'coordinates': {'top_left': {'x': 0.148, 'y': 0.462}, 'bottom_right': {'x': 0.429, 'y': 0.887}}, 'type': 'text'}, {'field_name': 'money_in', 'coordinates': {'top_left': {'x': 0.30778, 'y': 0.690323}, 'bottom_right': {'x': 0.345398, 'y': 1.02621}}, 'type': 'text'}, {'field_name': 'money_out', 'coordinates': {'top_left': {'x': 0.364776, 'y': 0.690323}, 'bottom_right': {'x': 0.402394, 'y': 1.02621}}, 'type': 'text'}, {'field_name': 'balance', 'coordinates': {'top_left': {'x': 0.427472, 'y': 0.690323}, 'bottom_right': {'x': 0.46509, 'y': 1.02621}}, 'type': 'text'}], 'row_delimiter': {'type': 'line'}}}
{'field_name': 'description', 'coordinates': {'top_left': {'x': 0.148, 'y': 0.462}, 'bottom_right': {'x': 0.429, 'y': 0.887}}, 'type': 'text'}
[0

In [14]:
template

{'metadata': {'template_id': 'barclays_statement_template',
  'version': '1.0.0'},
 'rules': [{'rule_id': 'sort_code',
   'type': 'form',
   'config': {'field_name': 'sort_code',
    'coordinates': {'top_left': {'x': 0.639, 'y': 0.427},
     'bottom_right': {'x': 0.705, 'y': 0.438}},
    'type': 'text'}},
  {'rule_id': 'account_number',
   'type': 'form',
   'config': {'field_name': 'account_number',
    'coordinates': {'top_left': {'x': 0.550299, 'y': 0.604435},
     'bottom_right': {'x': 0.602451, 'y': 0.620161}},
    'type': 'text'}},
  {'rule_id': 'customer_name',
   'type': 'form',
   'config': {'field_name': 'customer_name',
    'coordinates': {'top_left': {'x': 0.412653, 'y': 0.679839},
     'bottom_right': {'x': 0.550299, 'y': 0.697984}},
    'type': 'text'}},
  {'rule_id': 'start_balance',
   'type': 'form',
   'config': {'field_name': 'start_balance',
    'coordinates': {'top_left': {'x': 0.61271, 'y': 0.267742},
     'bottom_right': {'x': 0.661157, 'y': 0.283871}},
    'type

In [15]:
lines = pdf_data["pages"][page_number - 1]["lines"]

y_coordinates = []

for line in lines:
    x0 = line["decimal_coordinates"]["top_left"]["x"]
    x1 = line["decimal_coordinates"]["bottom_right"]["x"]
    y0 = line["decimal_coordinates"]["top_left"]["y"]
    y1 = line["decimal_coordinates"]["bottom_right"]["y"]
    max_coordinates = {
                            "top_left": {
                                "x": 0.148,
                                "y": 0.462
                            },
                            "bottom_right": {
                                "x": 0.429,
                                "y": 0.887
                            }
    }
    x_min = max_coordinates["top_left"]["x"]
    x_max = max_coordinates["bottom_right"]["x"]
    y_min = max_coordinates["top_left"]["y"]
    y_max = max_coordinates["bottom_right"]["y"]

    if y0 > y_min and y1 < y_max and x0 > x_min and x1 < x_max:
        y_coordinates.append(y0)

print(sorted(list(set(y_coordinates))))



[0.475534, 0.503682, 0.54323, 0.5877669999999999, 0.63076, 0.668765, 0.6720900000000001, 0.70677, 0.749762, 0.787767, 0.825772, 0.863777]


In [16]:
jpg_image = ImageDrawer.create_jpg_image(pdf_path, page_number)
image_drawer = ImageDrawer(jpg_image, jpg_image.size[0], jpg_image.size[1])


coordinates = [line["decimal_coordinates"] for line in lines]

image_copy = image_drawer.draw_coordinates(coordinates)

image_copy.show()



In [17]:
from src.extractor import Extractor

with open(pdf_path, "rb") as pdf_file:
    pdf_bytes = pdf_file.read()

extractor = Extractor(pdf_bytes)

pdf_jpg_files = extractor.convert_pdf_to_jpg_files(prefix="barclays_student_march")



AttributeError: 'Extractor' object has no attribute 'convert_pdf_to_jpg_files'

In [7]:
from PIL import Image, ImageDraw
import numpy as np
import io
import os

from src.extractor import Extractor, ImageExtractor

pdf_path: str = os.path.join(
    "data", "bank_statements", "barclays_student", "pdf", "barclays_student_march.pdf"
)


def create_average_color_image(average_pixel_value, size=(50, 50)):
    # Create an image filled with the average pixel color
    avg_color_image = Image.new("RGB", size, tuple(map(int, average_pixel_value)))
    return avg_color_image


def show_average_color_image(average_pixel_value, region, image, box_coords):

    # Draw a red box around the coordinates
    draw = ImageDraw.Draw(image)
    draw.rectangle(box_coords, outline="red", width=3)

    # Show the image with the red box
    image.show(title="JPEG File with Red Box")

    # Create an image of the average color
    average_color_image = create_average_color_image(average_pixel_value)

    # Display the original region and the average color image
    region_image = Image.fromarray(region)
    region_image.show(title="Extracted Region")
    average_color_image.show(title="Average Color Image")


with open(pdf_path, "rb") as pdf_file:
    pdf_bytes = pdf_file.read()

image_extractor = ImageExtractor(pdf_bytes)
pdf_jpg_files = image_extractor.convert_pdf_to_jpg_files(
    prefix="barclays_student_march"
)


with open(pdf_path, "rb") as pdf_file:
    pdf_bytes = pdf_file.read()

extractor = Extractor(pdf_bytes)

pdf_data = extractor.extract_text()


page_number = 2
lines = pdf_data["pages"][page_number]["lines"]

for line in lines[:1]:

    coordinates = line["decimal_coordinates"]

    jpg_bytes = pdf_jpg_files["barclays_student_march_page_2.jpg"]

    print(coordinates)

    average_pixel_value, region, image, box_coords = (
        image_extractor.calculate_average_pixel_value(jpg_bytes, coordinates)
    )

    show_average_color_image(average_pixel_value, region, image, box_coords)



{'top_left': {'x': 0.095126, 'y': 0.094418}, 'bottom_right': {'x': 0.152605, 'y': 0.094418}}
(np.float64(255.0), np.float64(255.0), np.float64(255.0))
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]
 

In [2]:
lines

[{'decimal_coordinates': {'top_left': {'x': 0.095126, 'y': 0.094418},
   'bottom_right': {'x': 0.152605, 'y': 0.094418}}},
 {'decimal_coordinates': {'top_left': {'x': 0.152269,
    'y': 0.11579600000000001},
   'bottom_right': {'x': 0.181176, 'y': 0.11579600000000001}}},
 {'decimal_coordinates': {'top_left': {'x': 0.18084, 'y': 0.11579600000000001},
   'bottom_right': {'x': 0.423529, 'y': 0.11579600000000001}}},
 {'decimal_coordinates': {'top_left': {'x': 0.423193,
    'y': 0.11579600000000001},
   'bottom_right': {'x': 0.514622, 'y': 0.11579600000000001}}},
 {'decimal_coordinates': {'top_left': {'x': 0.514286,
    'y': 0.11579600000000001},
   'bottom_right': {'x': 0.605042, 'y': 0.11579600000000001}}},
 {'decimal_coordinates': {'top_left': {'x': 0.604706,
    'y': 0.11579600000000001},
   'bottom_right': {'x': 0.695462, 'y': 0.11579600000000001}}},
 {'decimal_coordinates': {'top_left': {'x': 0.152269, 'y': 0.094418},
   'bottom_right': {'x': 0.695462, 'y': 0.094418}}},
 {'decimal_coo